## Creating sub-agent 2: Get technical analysis of the financial data

In this folder we will create the second sub-agent for collecting technical analysis to user questions. This agent will call the financial data API to handle the following user questions:

1. Questions about getting stock prices. Based on a user question, ticker over a given data range and interval, the agent will get the stock prices.

1. Questions about seeking information on the current stock price. If the user asks for the current stock price, then this agent will call a lambda function to get the current (latest) stock price for a ticker

1. If a user question is about calculating technical indicators for a given ticker and a time period, for example `RSI`, `MACD`, `SMA`, `EMA` and `Bollinger Bands` calculations.

### Prerequisites
---

You are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

Before running this notebook, make sure to populate the `.env` file with your `FINANCIAL_DATASETS_API_KEY` and `TAVILY_API_KEY`. These two `API` keys will be used by the agent to access information on user related questions. You can create an `API` here: https://www.financialdatasets.ai/

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

### Restart kernel

If you face issues to apply the latest multi-agent capabilities, uncomment this line to restart kernel to ensure packages updates to take effect

In [ ]:
import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
!pip freeze | grep boto3

### Importing helper functions

On following section, we're adding `agents.py` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `agent.py`.

Those files contain helper classes totally focused on make labs experience smoothly.

All interactions with Bedrock will be handled by these classes.

#### Following are methods that you're going to invoke on this lab:

- `create_agent`: Create a new agent and respective IAM roles

- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent

- `create_agent_alias`: Create an alias for this agent

- `invoke`: Execute agent

In [ ]:
import boto3
import os
import json
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

In [ ]:
import os
import sys
import time
import json
import boto3
import shutil
import logging
import zipfile
import subprocess
from dotenv import load_dotenv

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from utils import *
from globals import *
from utils.utils import *

In [ ]:
# Load environment variables from .env file
load_dotenv

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Defining agent configuration
---

In [ ]:
agent_instruction = """You are a technical analysis assistant that helps users analyze stock price movements and technical indicators. You can calculate and interpret various technical indicators using historical price data.

You have access to the following technical analysis capabilities based on the API key you have access to to call the available functions and get the following information:

1. Stock Price Data:
   - Current stock prices
   - Historical price data with various intervals
   - Custom date range analysis

2. Technical Indicators:
   - RSI
   - MACD
   - SMA
   - EMA
   - Bollinger Bands

You require the user to provide:
1. A stock ticker symbol
2. The type of technical indicator they want to analyze
3. Optionally: specific time periods or date ranges

Available functions:
1. get_current_stock_price: Retrieve latest stock price
2. get_stock_prices: Get historical price data for a specified period
3. get_technical_indicators: Calculate technical indicators for analysis

If you do not have access to the data that the user is asking for, do not make up an answer. Be completely accurate and only provide analysis based on the available technical indicators and price data.

Only answer questions related to technical analysis and price data based on the provided functions. If unsure, acknowledge limitations"""

agent_description = "Agent for technical analysis of stocks using various technical indicators and price data"

In [ ]:
# This is the function definition for the lambda function that will be invoked
# as a part of the action group
functions = [{
    'name': 'get_stock_prices',
    'description': 'Get prices for a ticker over a given date range and interval.',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "start_date": {
            "description": "Start date to get the stock price from",
            "required": True,
            "type": "string"
        },
        "end_date": {
            "description": "End data until which the stock price needs to be computed",
            "required": True,
            "type": "string"
        }, 
        "limit": {
            "description": "number of statements to retrieve",
            "required": True,
            "type": "integer"
        }
    }
},
{
    'name': 'get_current_stock_price',
    'description': 'Get the current (latest) stock price for a ticker.',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        }
    }
},
{
    'name': 'get_technical_indicators',
    'description': 'Calculate technical indicators (RSI, MACD, SMA, EMA, or Bollinger Bands) for a given ticker.',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "indicator": {
            "description": "technical indicator type (RSI, MACD, SMA, EMA, or BBANDS)",
            "required": True,
            "type": "string"
        },
        "period": {
            "description": "period for indicator calculation (default: 14)",
            "required": False,
            "type": "integer"
        },
        "start_date": {
            "description": "start date for analysis (YYYY-MM-DD)",
            "required": False,
            "type": "string"
        },
        "end_date": {
            "description": "end date for analysis (YYYY-MM-DD)",
            "required": False,
            "type": "string"
        }
    }
}]

### Add a lambda layer to the lambda function
---

In this section of the notebook, we will add a lambda layer to the lambda function. We will be installing `requests`, `ta` to make requests to the financial data API key.

In [ ]:
# Create and publish the layer ~ this step will take around 2 minutes

# In this case we want to add a layer to the lambda containing files to import the 
# requests, ta and pandas libraries
layer_zip = create_lambda_layer(['requests'])
layer_arn = publish_layer('technical-agent-lambda-layer-new')

### Create agent
---

Next we will create the agent with the provided information. In this, we will use the prompt configuration, the lambda function and lambda layers, update the lambda function to contain the `FINANCIAL_DATASETS_API_KEY` that the agent can use to fetch data to user inputs, and then finally create the agent.

In [ ]:
if region == "us-west-2":
    NOVA_LITE = f"us.{NOVA_LITE}"
    logger.info(f"Using the cross region inference model id for the technical analyst agent: {NOVA_LITE}")

# Update your agent creation code to include the action_group_config
technical_analyst_agent = agents.create_agent(
    SUB_AGENT_NAME_TECHNICAL_ANALYST,
    agent_description,
    agent_instruction,
    NOVA_LITE, # change to "us.<model-id> for cross region inference if you are using the agent in us.west.2"
)

In [ ]:
# Add the action group with lambda to this fundamental analyst agent
agents.add_action_group_with_lambda(
    agent_name=SUB_AGENT_NAME_TECHNICAL_ANALYST,
    lambda_function_name=f'{SUB_AGENT_NAME_TECHNICAL_ANALYST}-lambda',
    source_code_file=TECHNICAL_LAMBDA_FUNCTION_NAME,
    agent_functions=functions,
    agent_action_group_name="TechnicalAgentActionGroup",
    agent_action_group_description="Action group for technical analysis of stocks using various technical indicators and price data",
    lambda_layers=[layer_arn]
)

# Create a Lambda client and attach the API key as env variable to the lambda function
lambda_client = boto3.client('lambda')
lambda_function_name = f'{SUB_AGENT_FUNDAMENTAL_ANALYST}-lambda'
environment_variables = {
    'FINANCIAL_DATASETS_API_KEY': os.getenv('FINANCIAL_DATASETS_API_KEY')
}

In [ ]:
# Update the Lambda function's configuration to include the environment variables
# In the case of this agent, we need the financial dataset API as an env variable passed to the lambda
# so that it can be used in fetching data based on user questions
response = lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Environment={
        'Variables': environment_variables
    },
    Layers=[layer_arn]
)

### Testing the Fundamental Analyst agent
---

Now that we've created the agent, let's test it by using our `invoke_agent_helper` function. Here, we will invoke the `technical analyst` agent to provide information on `technical indicators` and `stock price data` based on the question provided by the user.

Since the `invoke_agent_helper` is wrapped with a `weave` decorator, it will log the input/outputs or any errors to your weave account dashboard. To create a weave API key, refer to the following link: https://wandb.ai/site/weave/

In [ ]:
# Sleep for 30 seconds before invoking the technical analyst agent
time.sleep(30)

In [ ]:
%%time
response = agents.invoke(
    """What is the latest stock price for AAPL from january 2022 to december 2022?""", 
    technical_analyst_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
%%time
response = agents.invoke(
    """What is the latest SMA for AAPL in the first 20 days of the year 2021?""", 
    technical_analyst_agent[0], enable_trace=True
)
print("====================")
print(response)

### Create alias

For multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional.

Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
technical_analyst_agent_id, technical_analyst_agent_alias_arn = agents.create_agent_alias(
    technical_analyst_agent[0], 'v1'
)

In [ ]:
techical_analyst_agent_arn = agents.get_agent_arn_by_name(SUB_AGENT_NAME_TECHNICAL_ANALYST)

In [ ]:
%store technical_analyst_agent_id
%store technical_analyst_agent_alias_arn
%store techical_analyst_agent_arn